### INLA for a hierarchical model (tennis example)

### Model

The purpose of this notebook is to try to get a version of INLA working on a simple hierarchical model.

The modelling question is to try to find the rate at which a player in tennis wins points on serve. In each match $i$, a player will serve on $n_i$ points, of which they win $p_i$. We'll treat this as a Binomial:

$p_i \sim \textrm{Binom}(n_i, \theta_i)$

where $\theta_i$ is the probability of winning a point on serve.

The model idea is that $\theta_i$ depends on both the server's skill at serving, $\alpha_{s(i)}$ (where $s(i)$ is the index of the server in match $i$) and the returner's skill at returning, $\beta_{r(i)}$.

We'll model this as a linear model. As in the INLA paper, we define the "linear predictor" $\eta_i$:

$\eta_i = \mu + \alpha_{s(i)} - \beta_{r(i)}$

Intuitively, a high serve skill $\alpha_{s(i)}$ will raise the server's chances of winning the point, while a high return skill $\alpha_{r(i)}$ will lower their chances.

We then pass this through a link function; here we use the probit link, so:

$\theta_i = \Phi(\eta_i)$

#### Data

Here, we extract what we need from the data.

In [3]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [4]:
data = pd.read_csv('dataset.csv', index_col=0)
data['tourney_date'] = pd.to_datetime(data['tourney_date'])

/Users/ingramm/miniconda3/envs/tf/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Clean dataset and keep only 2020 for speed
data = data.dropna(subset=['pts_won_serve_winner', 'pts_won_serve_loser', 
                           'pts_played_serve_winner', 'pts_played_serve_loser'])
data = data[(data['pts_played_serve_winner'] > 10) & (data['pts_played_serve_loser'] > 10)]
to_use = data[data['tourney_date'].dt.year >= 2020]

In [6]:
# Find the relevant quantities from the data
ps = np.concatenate([to_use['pts_won_serve_winner'], to_use['pts_won_serve_loser']])
ns = np.concatenate([to_use['pts_played_serve_winner'], to_use['pts_played_serve_loser']])
players = np.concatenate([to_use['winner_name'], to_use['loser_name']])
opponents = np.concatenate([to_use['loser_name'], to_use['winner_name']])

encoder = LabelEncoder()
p_ids = encoder.fit_transform(players)
opp_ids = encoder.transform(opponents)

n_m, n_p = ps.shape[0], len(encoder.classes_)

In [7]:
ps.min(), ns.min()

(11.0, 30.0)

### INLA

We now define the things we need for INLA.

Expressed the same way as in their paper, the model is:

$\pmb{\eta} = \mu \mathbf{1} + \mathbf{A_1}\mathbf{f_1} + \mathbf{A_2}\mathbf{f_2} + \pmb{\epsilon}$

Here, $\mu$ is the intercept, as before; $\mathbf{f_1}$ contains all the player serve skills, and $\mathbf{f_2}$ contains all the player return skills. $\pmb{\epsilon}$ is a noise term.

In [8]:
import scipy.sparse as sps

In [9]:
# Indicator matrices A_1 and A_2 picking out the match serve & return skills
A_1 = np.zeros((n_m, n_p))
A_1[np.arange(n_m), p_ids] = 1

A_2 = np.zeros((n_m, n_p))
A_2[np.arange(n_m), opp_ids] = -1

A_1, A_2 = map(sps.csc_matrix, [A_1, A_2])

Setting $\tau_{\epsilon}$, the precision of the noise term, is particularly tricky. We want it to be as large as possible, meaning that the artificial noise term $\epsilon$ is negligible. However, the computations appear to become numerically difficult if it is too large. $10^4$ or $10^5$ appears to work OK, but I'm a bit concerned that's not big enough.

We'll set $\tau_\mu$, the prior precision of the intercept, to $1$.

The parameters $\pmb{\theta}$ will be given by $\tau_{\alpha}$ and $\tau_{\beta}$. They form the hierarchical prior for each player's serve and return skills:

$\alpha_i \stackrel{iid}{\sim} \mathcal{N}(0, \tau_\alpha^{-1})$

$\beta_i \stackrel{iid}{\sim} \mathcal{N}(0, \tau_\beta^{-1})$

In other words, the prior precision matrices $Q_1$ and $Q_2$ will be diagonal with entries $\tau_\alpha$ and $\tau_\beta$ on the diagonal.

In [50]:
# Set parameters
# How to set tau_eps?
tau_eps = 10**4
tau_mu = 1.

# These two will be theta.
tau_alpha = 2.
tau_beta = 2.

In [51]:
from binomial_probit import likelihood_grad, likelihood_diag_hess, likelihood
from sksparse.cholmod import cholesky

In [52]:
# This mode-finding routine runs Newton-Raphson
# When it finds the mode, it returns the log marginal likelihood
def find_mode(Q_mat, maxiter=100, tol=1e-8):

    mu = np.zeros(Q_mat.shape[0])

    for i in range(maxiter):

        eta_mean = mu[:n_m]

        # Compute b and c
        b = likelihood_grad(eta_mean, ns, ps)
        c = likelihood_diag_hess(eta_mean, ns, ps)

        # Try the update
        c_full = np.zeros(Q_mat.shape[0])
        b_full = np.zeros(Q_mat.shape[0])

        b_full[:n_m] = b
        c_full[:n_m] = c

        # Negative Hessian
        solve_lhs = -(-Q_mat + sps.diags(c_full))

        # Jacobian (not negative)
        solve_rhs = -Q_mat.dot(mu) + b_full
        
        # R & W idea
        #solve_rhs = -c_full * mu + b_full

        cho_factor = cholesky(solve_lhs)
        update = cho_factor.solve_A(solve_rhs)
        #new_mu = cho_factor.solve_A(solve_rhs)
        
        #result_alt = np.linalg.solve(Q_mat.todense() - np.diag(c_full), solve_rhs)   
        #print(np.allclose(result_alt, update))
    
        new_mu = mu + update
        diff = np.linalg.norm(new_mu - mu)
        mu = new_mu

        if diff <= tol:
            # Double check this -- should it be negative or positive?
            logdet = -0.5 * cho_factor.logdet()
            post_term = -0.5 * mu.dot(Q_mat.dot(mu)) + likelihood(mu[:n_m], ns, ps)
            marg_lik = logdet + post_term
            
            # Prior logdet:
            prior_logdet = cholesky(Q_mat).logdet()
            marg_lik = marg_lik + 0.5 * prior_logdet
            
            #print(diff)
            return (mu, cho_factor, marg_lik, solve_lhs)
            
    raise Exception(f'Failed to converge after {maxiter} iterations')

In [53]:
# This routine assembles the big Q matrix of the GMRF
# Note the differences with the paper!
def make_q_mat(tau_alpha, tau_beta, n_p, A_1, A_2, tau_mu, tau_eps):
    
    Q_1 = sps.eye(n_p) * tau_alpha
    Q_2 = sps.eye(n_p) * tau_beta

    ones = sps.csc_matrix(np.ones((n_m, 1)))
    
    Q11 = tau_eps * sps.eye(n_m)
    Q12 = tau_eps * A_1
    Q13 = tau_eps * A_2
    Q14 = tau_eps * ones
    
    # Difference with paper: A_1.T A_1, not A_1 A_1.T!
    Q21 = Q12.transpose()
    Q22 = Q_1 + tau_eps * A_1.transpose().dot(A_1)
    Q23 = tau_eps * A_1.transpose().dot(A_2)
    Q24 = tau_eps * A_1.transpose().dot(ones)
    
    Q31 = Q13.transpose()
    Q32 = Q23.transpose()
    Q33 = Q_2 + tau_eps * A_2.transpose().dot(A_2)
    Q34 = tau_eps * A_2.transpose().dot(ones)
    
    Q41 = Q14.transpose()
    Q42 = Q24.transpose()
    Q43 = Q34.transpose()
    Q44 = tau_mu + tau_eps * np.sum(ones)
    
    # Difference with paper: had to make the negative signs!
    Q_mat = sps.bmat([[Q11, -Q12, -Q13, -Q14],
                      [-Q21, Q22, Q23, Q24],
                      [-Q31, Q32, Q33, Q34],
                      [-Q41, Q42, Q43, Q44]], format='csc')
            
    return Q_mat

In [55]:
# We can see if this works -- can we find the mode?
# If it doesn't crash, we're OK.

print('Theta:')
print(tau_alpha, tau_beta)

Q_mat = make_q_mat(tau_alpha, tau_beta, n_p, A_1, A_2, tau_mu, tau_eps)

mu, cho_factor, marg_lik, neg_hess = find_mode(Q_mat)

eta_star, f_1_star, f_2_star, mu_star = (np.split(mu, (n_m, n_m + n_p, n_m + 2 * n_p)))

print('Intercept is given by:')
mu_star

Theta:
2.0 2.0
Intercept is given by:


array([0.350799])

The routine above finds the mode of $\log p(\mathbf{x} | \mathbf{y}, \pmb{\theta})$ and uses the Gaussian approximation to compute $\log p(\mathbf{y} | \pmb{\theta})$, the marginal likelihood of the parameters $\pmb{\theta}$  having integrated out the latent field $\mathbf{x}$.

We now use numerical optimisation to locate the mode of the approximation to $\log p(\mathbf{y} | \pmb{\theta})$:

In [56]:
def to_optimise(x):
    
    tau_alpha = np.exp(x[0])
    tau_beta = np.exp(x[1])
    
    Q_mat = make_q_mat(tau_alpha, tau_beta, n_p, A_1, A_2, tau_mu, tau_eps)
    
    neg_marg_lik = -find_mode(Q_mat)[-2]
    
    print(tau_alpha, tau_beta, neg_marg_lik)
    
    return neg_marg_lik

In [57]:
from scipy.optimize import minimize
result = minimize(to_optimise, [0, 0])

1.0 1.0 68216.53170466259
1.0 1.0 68216.53170466259
1.0000000149011612 1.0 68216.5317032147
1.0 1.0000000149011612 68216.5317032142
2.042257546936305 2.042755926510594 68080.23816347733
2.042257546936305 2.042755926510594 68080.23816347733
2.0422575773683143 2.042755926510594 68080.23816208894
2.042257546936305 2.04275595695003 68080.23816208594
35.52651017127685 35.56987965402855 67672.94880992346
35.52651017127685 35.56987965402855 67672.94880992346
35.5265107006631 35.56987965402855 67672.9488094817
35.52651017127685 35.56988018406106 67672.94880932153
0.12392798229901394 2422359.0901394696 68264.64197040712
0.12392798229901394 2422359.0901394696 68264.64197040712
0.1239279841456848 2422359.0901394696 68264.64196890181
0.12392798229901394 2422359.126235433 68264.6419704088
21.923807849405954 91.91649255120439 67669.70666192769
21.923807849405954 91.91649255120439 67669.70666192769
21.923808176096152 91.91649255120439 67669.70666117771
21.923807849405954 91.91649392086686 67669.70666

74.29265313973248 118.70744568379732 67637.18012841554
74.29265424677929 118.70744568379732 67637.18012841865
74.29265313973248 118.7074474526761 67637.180128417
74.29265313973248 118.70744568379732 67637.18012841554
74.29265313973248 118.70744568379732 67637.18012841554
74.29265424677929 118.70744568379732 67637.18012841865
74.29265313973248 118.7074474526761 67637.180128417
74.29265313973241 118.70744568379732 67637.18012841647
74.29265313973241 118.70744568379732 67637.18012841647
74.29265424677922 118.70744568379732 67637.18012841865
74.29265313973241 118.7074474526761 67637.1801284173
74.29265313973248 118.70744568379732 67637.18012841554
74.29265313973248 118.70744568379732 67637.18012841554
74.29265424677929 118.70744568379732 67637.18012841865
74.29265313973248 118.7074474526761 67637.180128417
74.29265313973248 118.70744568379732 67637.18012841554
74.29265313973248 118.70744568379732 67637.18012841554
74.29265424677929 118.70744568379732 67637.18012841865
74.29265313973248 118

In [44]:
# The routine finishes, but note that it complains about not having succeeded.
result

      fun: 67637.18012841597
 hess_inv: array([[ 0.00783317, -0.00136234],
       [-0.00136234,  0.00023743]])
      jac: array([-0.0625    , -0.02148438])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 210
      nit: 7
     njev: 50
   status: 2
  success: False
        x: array([4.30801204, 4.77666203])

In [45]:
tau_alpha = np.exp(result.x[0])
tau_beta = np.exp(result.x[1])

Q_mat = make_q_mat(tau_alpha, tau_beta, n_p, A_1, A_2, tau_mu, tau_eps)

mu, cho_factor, marg_lik, neg_hess = find_mode(Q_mat)

eta_star, f_1_star, f_2_star, mu_star = (np.split(mu, (n_m, n_m + n_p, n_m + 2 * n_p)))

In [46]:
mu_star

array([0.3429802])

#### Inspecting the results

In [47]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd


In [48]:
# Keep only players with more than five matches to avoid noisy estimates:
player_names = to_use['winner_name'].tolist() + to_use['loser_name'].tolist()
to_keep = pd.Series(player_names).value_counts()
to_keep = to_keep[to_keep > 5].index

# Top 10 servers
serve_skills = pd.Series(f_1_star, index=encoder.classes_)
serve_skills[to_keep].sort_values(ascending=False).head(10)

Reilly Opelka         0.279668
Nick Kyrgios          0.271313
Stefanos Tsitsipas    0.268478
Milos Raonic          0.254906
Novak Djokovic        0.251695
John Isner            0.231733
Rafael Nadal          0.201243
Andrey Rublev         0.180117
Stan Wawrinka         0.172001
Casper Ruud           0.161920
dtype: float64

In [49]:
# Top 10 returners
ret_skills = pd.Series(f_2_star, index=encoder.classes_)
ret_skills[to_keep].sort_values(ascending=False).head(10)

Novak Djokovic           0.238196
Roberto Bautista Agut    0.177184
Rafael Nadal             0.171545
Gael Monfils             0.166033
Diego Schwartzman        0.163977
David Goffin             0.154151
Daniil Medvedev          0.138422
Andrey Rublev            0.129488
Stefanos Tsitsipas       0.122404
Marton Fucsovics         0.121049
dtype: float64

#### Next steps

Still need to:

* Find the marginal variances of the latent field components $x_i$.
* Try to do the integration over different $\pmb{\theta}$ rather than just finding the mode, as we did here.